Кобзарь Олег, Хабибуллин Ринат, Буденный Семен, Адрианова Алла, Логинов Александр, Лемихов Александр
# Обработчик данных 

В данной тетрадке собраны все средства для работы с данными (пре- и пост- процессор; генерация входных данных для модели, анализ рассчитанных значений, построение графиков)

Большое количество кода вынесено в модуле - здесь в основном управление

`well_name` - номер исследуемой скважины, например `1`

Исходными данные лежат в папке  `data` в папках, соответствующим названиям скважин. Это данные
1. Со станции управления (`data/well_name/well_name.csv`). Большое количество записей, которые нужно преобразовать в "шахмоткоподобный" вид для дальнейшей работы. Данные со СУ могут быть как высокочастотными (токи, напряжения), так и низкочастотными. 
2. С шахматки `data/well_name/Скв. well_name (01.07.2018-31.03.2019).xls`. Низкочастотные данные. 
3. C техрежима `data/tr/Техрежим, , февраль 2019.xls`. Какой насос спущен, ПЭД, на какую глубину. Эти данные постоянны на всем периоде расчета. В одном файлике один месяц и данные для всех скважин.



## Общий workflow
Здесь кратко, подробное описание будет над ячейками.
1. Обработка исходных данных со СУ и приведение их в удобный формат. (init_edit)
2. Считавание подготовленных ранее данных со СУ, шахматки, а затем генерации входных данных для модели, конкретно, адаптации модели скважины за выбранный период времени. (adaptation_input)
3. Адаптация. Теперь можно открыть файл processor.py и в нем, выставив настройки и определив входные данные, запустить расчет скважину. После адаптации данные появится данные с калибровочными коэффициентами. (adaptation)
4. Считывание рассчитанных значений адаптации, данных со СУ и шахматки. Генерации файлов для восстановления дебитов (restore_input)
5. Восстановление. Работа в processor.py, где необходимо изменить метод расчета и указать на новые входные данные. (restore) 
6. Заключительный анализ: сведение результатов адаптации и восстановления; расчет различных метрик для оценки метода (тоже в папке restore)

По идее, если данные уже сгенерированы, каждый пункт может повторяться многократно

Импорт модулей. Некоторые функции и методы вынесены в отдельные `.py`
* `preprocessor` - для обработки данных в и из модели
* `processor` - для непосредственного запуска расчета адаптации или восстановления
* `postprocessor` - для завершающего анализа сгенерированных данных. Определение метрик успешности модели, паттернов, событий, зависимостей.
* `plotly_workflow` - для быстрого построения шаблонизированных графиков

In [ ]:
import os
import sys
sys.path.append('../../../')
import pandas as pd
import unifloc.uniflocpy.uTools.plotly_workflow as pw
import datetime
from multiprocessing import Pool

In [ ]:
import plot_workflow.plotly_option as pltl_opt
import plot_workflow.plotly_workflow as pltl_wf

from preproc_p import workflow_cs_data
from preproc_p import workflow_chess_data
from preproc_p import preproc_tool
from preproc_p import workflow_calc_data
from preproc_p import workflow_tr_data
from preproc_p import filtration
from proc_p import processor as proc

from ml import calibr_restore as calibr_restore
from postproc_p import result_and_metrics as result_and_metrics

Общие настройки и флаги для работы. Указание номера скважины и название файла шахматки. Флаги для запуска тех или иных ячеек

In [ ]:
well_name = '540'
chess_file_name = 'Скв. ' + well_name + ' (01.08.2018-28.02.2019).xls'
tr_name = "Техрежим, , февраль 2019.xls"
# TODO убрать флаги
read_initial_data = True
plot_initial_data = True
create_input_data = True
multiprocessing_on = True #TODO может быть убрать флаги, как лучше?
created_input_data_type = 0 # 0 - adaptation input, 1 - restore input with calibation coefficients

Определение путей к данным. Создание метки времени к генерируемым папкам. (чтоб данные не перезаписывались)

In [ ]:
current_path = os.getcwd()
time_mark = '' #datetime.datetime.today().strftime('%Y_%m_%d_%H_%M_%S')
path_to_data = current_path + "\\data\\"
path_to_work_dir = current_path + "\\data\\" + well_name +  "\\"
save_dir_name = 'init_edit'
path_to_save = path_to_work_dir + save_dir_name + '\\'
dirnames_list = []
for (dirpath, dirnames, filenames) in os.walk(path_to_data):
    dirnames_list.extend(dirnames)
    break
print(dirnames_list)
cs_data_filename = path_to_save + well_name + "_first_edit.csv"

Задание интервалов, на которых будет вестить расчет. Их может быть несколько. Рекомендуется поставить малый период времени, выявить ошибки, а затем запускать на месяцы и годы

In [ ]:
left_boundary = [datetime.datetime(2018,8,1), datetime.datetime(2018,11,29)]
right_boundary = [datetime.datetime(2018,11,5), datetime.datetime(2019,2,28)]
#left_boundary = [datetime.datetime(2018,8,3), datetime.datetime(2018,11,29),   datetime.datetime(2019,2,19)]
#right_boundary = [datetime.datetime(2018,11,6), datetime.datetime(2019,2,4),  datetime.datetime(2019,2,28)]
#left_boundary = [datetime.datetime(2019,1,30)]
#right_boundary = [datetime.datetime(2019,2,28)]
#left_boundary = [datetime.datetime(2018,8,1)]
#right_boundary = [datetime.datetime(2018,12,2)]

Определение функции которая все запустит в многопотоке и произведет из данного ноутбука адаптацию и восстановление. (Временный костыль - работает только здесь, в модуле не может)

In [ ]:
def run_calculation(thread_option_list):
    if __name__ == '__main__':
        with Pool(amount_of_threads) as p:
            p.map(proc.calc,
                  thread_option_list)

# 1. Обработка исходных данных
Чтение исходных данных `well_name.csv` со СУ и преобразование их в удобный формат. Процесс небыстрый из-за несовершенства алгоритма форматирования и количества данных (около 1-1,5 млн. записей)

In [ ]:
%%time
if read_initial_data:
    try:
        os.mkdir(path_to_work_dir + save_dir_name)
    except:
        pass
    well_data = workflow_cs_data.read_and_edit_init_cs_data(well_name, path_to_work_dir)

Сохранение обработанных данных со СУ в папку `init_edit`

Процесс может быть долгим - 3 минуты

In [ ]:
%%time
#TODO использовать библиотеку modin вместо pandas
well_data.to_csv(path_to_save + well_name + "_first_edit.csv")

Построение графиков для нескольких интересующих параметров по исходным данных со СУ в `well_name__first_edit_report.html`. Процесс занимает около 3-5 минут.

In [ ]:
%%time
well_data['ГФ, м3/м3'] = well_data['Объемный дебит газа'] / well_data['Объемный дебит нефти']
all_banches = pltl_opt.create_banches_for_report(well_data, report_type = 'init_cs_data')
pw.create_report_html(well_data, all_banches, path_to_save + well_name + "_first_edit_report.html")
del well_data['ГФ, м3/м3']

Построение исходных данных с шахматки в `well_name_chess_report.html`

In [ ]:
%%time
chess_data = workflow_chess_data.load_and_edit_chess_data(path_to_work_dir + chess_file_name,
                                     '1d', without_changing = True) #TODO сделать протяжку вверх, для буферного давления


all_banches = pltl_opt.create_banches_for_report(chess_data, report_type = 'init_chess_data')
pw.create_report_html(chess_data, all_banches, path_to_save + well_name + "_chess_report.html")


На этом этапе в папке init_edit окажутся
1. well_name_init_edit.csv - данные с шахматки, которые будут подгружаться для адаптации и восстановления
2. well_name_init_sp.html - график для исходных данных
3.  well_name_init_edit_report.html - график для выбранных исходных данных

Перейдем к следующему этапу

# 2. Генерация входных данных для адаптации

Для данного этапа уже нужные обработанные данные со станции управления.

Чтение исходных данных со СУ (в уже подготовленных). Обработка этих данных - ресемпл, пометка названий столбцов с помощью `(СУ)`. Т.к. данные высокочастотные, осредним их по 3 часам - примерному времени замера дебита скважины на АГЗУ. Обработка будет отличаться в зависимости от того, какие данные вы генерируете - для адаптации выбрасываются интервалы без замеров дебитов, для восстановления - нет

In [ ]:
time_to_resamle = '3h'
created_input_data_type = 0 

In [ ]:
%%time
edited_data_cs = workflow_cs_data.load_and_edit_cs_data(cs_data_filename,
                                       created_input_data_type,
                                           time_to_resamle)

Произведем фильтрацию

In [ ]:
edited_data_cs = filtration.get_filtred_by_sigma(edited_data_cs) #TODO добавить сюда фильтрацию по времени замера

Чтение данных с шахматки. Осреднение и протяжка параметров. Пометка названий столбцов маркером `(Ш)`

In [ ]:
%%time
chess_data = workflow_chess_data.load_and_edit_chess_data(path_to_work_dir + chess_file_name,
                                     time_to_resamle)

Генерация входных данных для адаптации. Сведение данных с шахматки и со СУ. Создание папки с названием `adapt_input_` и временной пометкой и помещение в нее:
* сгенерированных входных данных для модели `well_name_adapt_input.csv`

In [ ]:
input_data_dir_name = 'adapt_input_' + time_mark
path_to_input_data = path_to_work_dir + input_data_dir_name + '\\'
if create_input_data:
    try:
        os.mkdir(path_to_work_dir + input_data_dir_name)
    except:
        pass
created_input_data = edited_data_cs.join(chess_data, how = 'inner') #TODO протягивать буферное давление вверх
created_input_data = created_input_data.dropna(subset = ['Объемный дебит жидкости (СУ)'])
created_input_data = preproc_tool.cut_df(created_input_data, left_boundary, right_boundary)
created_input_data = preproc_tool.check_input_data(created_input_data)
created_input_data.to_csv(path_to_input_data + well_name + '_adapt_input.csv')

Все входные данные на графике `well_name_adapt_input.html` в той же папке

In [ ]:
plot_file_path = path_to_input_data + well_name + '_adapt_input.html'
input_data_traces = pw.create_traces_list_for_all_columms(created_input_data, 'lines+markers', use_gl = True)
pw.plot_subplots(input_data_traces, plot_file_path, True)

Построение кривых в отчете только для тех данных, которые будут использоваться в модели в `well_name_adapt_input_report.html` в той же папке в стандартном виде. Можно настроить, какие данные выводить.

In [ ]:
all_banches = pltl_opt.create_banches_for_report(created_input_data, report_type = 'adapt_input')

pw.create_report_html(created_input_data, all_banches, path_to_input_data + well_name + '_adapt_input_report.html')

# 3. Адаптация
Нужно, используя сгенерированные данные `well_name_adapt_input` получить значения коэффициентов калибровок по напору и мощности с помощью `processor.py`

Для этого выключить флаги

* vfm_calc_option = False
* restore_q_liq_only = False


Подготовка к параллельному расчету.

В проекте UniflocVBA нужно вручную размножить надстройки до нужного количества потоков - каждая надстройка будет работать параллельно и рассчитывать определенную часть общих данных. 

По умолчанию выставлено 4 потока на 4 надстройках `UniflocVBA_7.xlam`, `UniflocVBA_7_1.xlam`, `UniflocVBA_7_2.xlam`, `UniflocVBA_7_3.xlam`. При желании можно добавить еще, не забыв изменить также номера потоков и их общее количество.

Группировка информации о потоках в единый список `thread_option_list`

In [ ]:
# настройка многопоточности
amount_of_threads = 12
tr_name = "Техрежим, , февраль 2019.xls"
dir_name_with_input_data = 'adapt_input_'

thread_option_list =proc.create_thread_list(well_name, dir_name_with_input_data, tr_name,
                       amount_of_threads)

Запуск процесса адаптации - получения калибровок. Процесс может быть долгим - в среднем расчет идет медленнее, чем при восстановлении дебитов. 

Рассчет 1 месяца в среднем занимает около 15-20 минут.

Стоит отметить, что для аварийного выхода нужно вручную закрыть надстройки Excel (окна, которые открыты и в них ведется работа). После закрытия всех четырех процесс остановится и возникнет ошибка, контроль над jupyter notebook вернется и можно будет работать.

Поэтому для тщательной отладки `processor.py` рекомендуется запускать отдельно (лучше через PyCharm).

In [ ]:
%%time
run_calculation(thread_option_list)

# 4. Генерация данных для восстановления дебитов

## 4.1 Загрузка и анализ адаптации
После успешной адаптации нужно проанализировать ее корректность, построив графики. Для этого нужно указать директории с 
* результатами адапатации - `adaptation_time_mark`  
* входными данными для адаптации - `adaptation_input_time_mark`

In [ ]:
dir_name_with_input_data = 'adapt_input_' + '\\'
input_data_file_name = well_name + '_adapt_input'
dir_name_with_calculated_data = 'adaptation_' + '\\'
calculated_data_file_name = well_name + '_adapt_1'

Если расчет проводился в многопоточном варианте, то загрузка будет отличаться. В папке с расчетами `adaptation` в сабдиректории `multiprocessing` будут результаты по частям в разных файликах, поэтому их нужно собрать в один файлик и поместить на уровень выше в `adaptation`. Если расчет был без использования многопоточности, этот шаг можно пропустить.

In [ ]:
if multiprocessing_on == True:
    first_result_data = preproc_tool.combine_multiprocessing_result(path_to_work_dir, dir_name_with_calculated_data)
    first_result_data.to_csv(path_to_work_dir + dir_name_with_calculated_data + calculated_data_file_name + '.csv')

Загрузка данных адаптации после расчета - в папке `adaptation_time_mark`  файл `well_name_adapt_1.csv`

In [ ]:
calculated_data = workflow_calc_data.load_calculated_data_from_csv(path_to_work_dir + dir_name_with_calculated_data +
                                                calculated_data_file_name +  '.csv')

Чтение входных данных для адаптации из папки `adaptation_input` и объединение данных адатации и входных данных для нее.
Сохранение всех данных в папке `adaptation_time_mark` с названием `well_name_calc_and_input.csv`

In [ ]:
input_data = pd.read_csv(path_to_work_dir + dir_name_with_input_data + input_data_file_name +  '.csv')
input_data.index = input_data['Время']
del input_data['Время']
all_data = input_data.join(calculated_data, how = 'outer')

all_data.to_csv(path_to_work_dir + dir_name_with_calculated_data + well_name + '_calc_and_input' +  '.csv' )

Построение графиков для данных адаптации в папке  `adaptation_time_mark`

Все данные (входные и результаты) на одном графике `well_name_calc_and_input.html`

In [ ]:
adapt_data_traces = pw.create_traces_list_for_all_columms(all_data, 'lines+markers', use_gl = True)
plot_file_path = path_to_work_dir + dir_name_with_calculated_data + well_name + '_calc_and_input' +  '.html'
pw.plot_subplots(adapt_data_traces, plot_file_path, True)

Создание отчета - набор нужных графиков для быстрой оценки результатов адаптации

In [ ]:
all_banches = pltl_opt.create_banches_for_report(all_data, report_type = 'adapt_report')

pw.create_report_html(all_data, all_banches, path_to_work_dir + dir_name_with_calculated_data + 
                      well_name + '_adapt_report.html')

# 4.2 Генерации данных для восстановления дебитов
Работа с калибровками для восстановления дебитов. Для преобразования их выделим в отдельный DataFrame. Будем интерполировать, выкалавать точки, либо еще что-нибудь

In [ ]:
calibr_data = calculated_data[['К. калибровки по напору - множитель (Модель)',
                               'К. калибровки по мощности - множитель (Модель)']]
calibr_data = preproc_tool.mark_df_columns(calibr_data, 'Подготовленные')
#calibr_data = calibr_data.resample('3h').mean()
#calibr_data = calibr_data.interpolate()

In [ ]:
p_out_lol, f_out_lol = calibr_restore.restore_calibr_via_ridge(all_data, calibr_data, use_80_20 = True)

In [ ]:
time_to_resamle = '3h'
created_input_data_type = 1 # костыль для 252 и 693 скважины

In [ ]:
#TODO учесть вариант, при котором калибровки линейно интерполируются

In [ ]:
edited_data_cs = workflow_cs_data.load_and_edit_cs_data(cs_data_filename,created_input_data_type,
                                       time_to_resamle
                                       )
chess_data = workflow_chess_data.load_and_edit_chess_data(path_to_work_dir + chess_file_name,
                                     time_to_resamle)

Прозведем фильтрацию

In [ ]:
edited_data_cs = filtration.get_filtred_by_sigma(edited_data_cs) #TODO добавить сюда фильтрацию по времени замера

In [ ]:
result = preproc_tool.make_gaps_and_interpolate(calibr_data)
calibr_data = result.copy()
#all_data = all_data.join(calibr_data, how = 'inner')
#created_input_data = all_data.copy()

In [ ]:
# вставляем предсказания машины
calibr_data['К. калибровки по мощности - множитель (Модель) (Подготовленные)'] = p_out_lol
calibr_data['К. калибровки по напору - множитель (Модель) (Подготовленные)'] = f_out_lol

Для прогноза на хвосте обрежем данные, чтобы считать только прогнозные часть - 20% от всего

In [ ]:
calibr_data = calibr_data[calibr_data['К. калибровки по мощности - множитель (Модель) (Подготовленные)']!=
                          calculated_data['К. калибровки по мощности - множитель (Модель)']]

Прочитаем данные со СУ и шахматки и обработаем их, для генерации входных данных модели

Генерация входных данных для модели для восстановления дебитов в папку `restore_input_time_mark`
* `well_name_restore_input.csv` - входные данных с калибровками (увеличили частоту дискретизации или выкололи точки)
* `well_name_restore_input.html` - все входные данные на одном графике

In [ ]:
input_data_dir_name = 'restore_input_' + time_mark
path_to_input_data = path_to_work_dir + input_data_dir_name + '\\'
if create_input_data:
    try:
        os.mkdir(path_to_work_dir + input_data_dir_name)
    except:
        pass
created_input_data = edited_data_cs.join(chess_data, how = 'inner')
created_input_data = created_input_data.join(calibr_data, how ='inner')
created_input_data = preproc_tool.cut_df(created_input_data, left_boundary, right_boundary)
created_input_data.to_csv(path_to_input_data + well_name + '_restore_input.csv')

In [ ]:
plot_file_path = path_to_input_data + well_name + '_restore_input.html'
input_data_traces = pw.create_traces_list_for_all_columms(created_input_data, 'lines+markers', use_gl = True)
pw.plot_subplots(input_data_traces, plot_file_path, True)

Построение графиков в отчетной форме 

In [ ]:
all_banches = pltl_opt.create_banches_for_report(created_input_data, report_type = 'restore_input')

pw.create_report_html(created_input_data, all_banches, path_to_input_data  + 
                      well_name + '_restore_input_report.html')

# 5. Восстановление дебитов.
Нужно, используя сгенерированные данные `well_name_restore_input` восстановить дебиты с помощью `postprocessor.py`
Для этого активировать флаги
* vfm_calc_option = True
* restore_q_liq_only = True

Подготовка к параллельному расчету.

В проекте UniflocVBA нужно вручную размножить надстройки до нужного количества потоков - каждая надстройка будет работать параллельно и рассчитывать определенную часть общих данных. 

По умолчанию выставлено 4 потока на 4 надстройках `UniflocVBA_7.xlam`, `UniflocVBA_7_1.xlam`, `UniflocVBA_7_2.xlam`, `UniflocVBA_7_3.xlam`. При желании можно добавить еще, не забыв изменить также номера потоков и их общее количество.

Группировка информации о потоках в единый список `thread_option_list`

In [ ]:
# настройка многопоточности

amount_of_threads = 12
tr_name = "Техрежим, , февраль 2019.xls"
dir_name_with_input_data = 'restore_input_'

thread_option_list =proc.create_thread_list(well_name, dir_name_with_input_data, tr_name,
                       amount_of_threads)

Запуск процесса восстановления дебитов. Процесс может быть долгим - в среднем расчет восстановления дебитов идет быстрее, чем при адаптации (получении калибровок). 

Рассчет 1 месяца в среднем занимает около 10-15 минут.

Стоит отметить, что для аварийного выхода нужно вручную закрыть надстройки Excel (окна, которые открыты и в них ведется работа). После закрытия всех четырех процесс остановится и возникнет ошибка, контроль над jupyter notebook вернется и можно будет работать.

Поэтому для тщательной отладки processor.py рекомендуется его запускать отдельно.

In [ ]:
%%time
run_calculation(thread_option_list)

Результаты расчета появятся в `well_name/restore_time_mark/multiprocessing`

# 6. Работа с данными после восстановления дебитов

# 6.1 Первичная обработка результатов восстановления
Обозначим директории с результатами восстановления дебитов и входными данными

In [ ]:
dir_name_with_input_data = 'restore_input_' + '\\'
input_data_file_name = well_name +'_restore_input'
dir_name_with_calculated_data = 'restore_' + '\\'
calculated_data_file_name = well_name +'_restore_1'

Объединение данных при запуске расчета в многопотоке. Если расчет проводился без многопоточности, этот шаг можно пропустить.

In [ ]:
if multiprocessing_on == True:
    first_result_data = preproc_tool.combine_multiprocessing_result(path_to_work_dir, dir_name_with_calculated_data)
    first_result_data.to_csv(path_to_work_dir + dir_name_with_calculated_data + calculated_data_file_name + '.csv')

Загрузка данных после восстановления дебитов

In [ ]:
calculated_data = workflow_calc_data.load_calculated_data_from_csv(path_to_work_dir + dir_name_with_calculated_data +
                                                calculated_data_file_name +  '.csv')

Объединение входных и выходных данным модели. Сохранение результатов в `well_name_calc_and_input.csv`

In [ ]:
input_data = pd.read_csv(path_to_work_dir + dir_name_with_input_data + input_data_file_name +  '.csv', parse_dates = True, index_col = 'Время')
all_data = input_data.join(calculated_data, how = 'outer')
all_data.to_csv(path_to_work_dir + dir_name_with_calculated_data + well_name + '_calc_and_input' +  '.csv' )

Построение графиков восстановления дебитов

In [ ]:
input_data_traces = pw.create_traces_list_for_all_columms(all_data, 'lines+markers', use_gl = True)
plot_file_path = path_to_work_dir + dir_name_with_calculated_data + well_name + '_calc_and_input' +  '.html'
pw.plot_subplots(input_data_traces, plot_file_path, True)

# 6.2 Сведение данных адаптации и восстановления
Использование файлов типа `calc_and_input` в папках с адаптацией и восстновлением для формирования общего отчета

In [ ]:
path_to_adapt_dir = 'adaptation_' + '\\'
path_to_restore_dir = 'restore_' + '\\'

Загрузка и слияние данных адаптации и восстановления

In [ ]:
adapt_data_with_input = pd.read_csv(path_to_work_dir + path_to_adapt_dir + well_name + '_calc_and_input' + '.csv' , parse_dates = True, index_col = 'Время')
adapt_data_with_input = preproc_tool.mark_df_columns(adapt_data_with_input, 'ADAPT')
restore_data_with_input = pd.read_csv(path_to_work_dir + path_to_restore_dir + well_name + '_calc_and_input' + '.csv' , parse_dates = True, index_col = 'Время')
restore_data_with_input = preproc_tool.mark_df_columns(restore_data_with_input, 'RESTORE')
overall_data = adapt_data_with_input.join(restore_data_with_input, how = 'outer')

Получение дебитов методом линейной интерполяции

In [ ]:
q_liq = overall_data[['Объемный дебит жидкости (СУ) (ADAPT)', 'Активная мощность (СУ) (ADAPT)']]
result = preproc_tool.make_gaps_and_interpolate(q_liq)
result = preproc_tool.mark_df_columns(result, 'INTERP')
overall_data = overall_data.join(result)

Заключительная обработка данных

In [ ]:
overall_data = result_and_metrics.final_edit_overall_data(overall_data)

Подготовка вывода в отчет

In [ ]:
all_banches = pltl_opt.create_banches_for_report(overall_data, report_type = 'overall_result')


Создание сводного отчета `well_name_adapt_and_restore_report.html`

In [ ]:
plot_file_path = path_to_work_dir + path_to_restore_dir + well_name + '_adapt_and_restore_report' +  '.html'
pw.create_report_html(overall_data, all_banches, plot_file_path)

Расчет различных метрик для модели и сохранение их в текстовый файл `well_name_adapt_and_restore_metrics_report.txt`

In [ ]:
calibr_calc_metrics, interp_calc_metrics = result_and_metrics.calc_calibr_interp_metrics(overall_data)

In [ ]:
metrics_text_file_path = path_to_work_dir + path_to_restore_dir + well_name + '_adapt_and_restore_metrics_report' +  '.txt'
text_file = open(metrics_text_file_path, "w")
text_file.write(calibr_calc_metrics + '\n' + interp_calc_metrics)
text_file.close()

## 6.3 Построение тепловой карты, характеристик и обезразмеренных величин
Пару дополнительных функций

Указание имени файла с техрежимов (при ошибке чтения его нужно открыть и закрыть)

In [ ]:
tr_name = "Техрежим, , февраль 2019.xls"

Чтение данных с техрежима для данной скважины

In [ ]:
tr_file_full_path = os.getcwd() + '\\data\\tr\\' + tr_name
tr_data = workflow_tr_data.read_tr_and_get_data(tr_file_full_path, well_name)
tr_data_df = pd.DataFrame({'Параметры скважины с ТР': list(tr_data.__dict__.values())})
tr_data_df.index = list(tr_data.__dict__.keys())

Объединение метрик и данных по скважине в единый DataFrame (вместе с линейной интерполяцией). Сохранение в `_adapt_restore_metrics_tr_report`

In [ ]:
overall_metrics = result_and_metrics.calc_calibr_interp_metrics(overall_data, return_df = True)

overall_metrics['ЭЦН'] = [tr_data.esp_name_str, tr_data.esp_name_str]
overall_metrics['ПЭД'] = [tr_data.motor_name_str, tr_data.motor_name_str]
overall_metrics['Кол-во точек (ADAPT)'] = [len(overall_data['К. калибровки по напору - множитель (Модель) (ADAPT)']), 
                                           len(overall_data['К. калибровки по напору - множитель (Модель) (ADAPT)'])]

overall_metrics.index = [well_name + ' (CALIBR)' , well_name + ' (INTERP)']
overall_metrics.to_csv(path_to_work_dir + path_to_restore_dir + well_name + '_adapt_restore_metrics_tr_report' +  '.csv')
overall_metrics.head()

Вызов UniflocVBA и построение напорных характеристик (по воде) данного насоса, создание общего отчета с пометкой `_dimless_pump_heatmap_report`

In [ ]:
q_esp_nom_m3day = tr_data.esp_nom_rate_m3day
head_esp_nom_m = tr_data.esp_nom_head_m

import unifloc_vba.description_generated.python_api as python_api
path_to_addin = os.getcwd()
path_to_addin = path_to_addin.replace('unifloc\\sandbox\\uTools', 'unifloc_vba\\UniflocVBA_7.xlam')
UniflocVBA = python_api.API(path_to_addin)
esp_traces = pltl_wf.create_esp_traces(UniflocVBA, q_esp_nom_m3day, head_esp_nom_m)
overall_data_dimensionless = result_and_metrics.make_dimensionless_df(overall_data)
filename = path_to_work_dir + path_to_restore_dir + well_name + '_dimless_pump_heatmap_report' + '.html'
pltl_wf.create_overall_report(overall_data, overall_data_dimensionless, esp_traces, filename)

# Конец